In [2]:
import numpy as np
import pandas as pd
import six
import time
import os
from tqdm import tqdm
import pandas as pd
import bs4

import random
import string

# Load raw scraped dataset

In [3]:
scraped_data = pd.read_pickle('data/scraped/scrape.pickle.gz', compression='gzip')
print(scraped_data.shape)

(130348, 5)


In [4]:
scraped_data.head()

,name,image_path,price,url,raw_html
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
4,Clarendon Hills Astralis Syrah 2011,/product/images/fl_progressive/qajrqr4d6ttn2pf...,149.99,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


# Load cleaned descriptions

In [7]:
descriptions = pd.read_csv(
    "data/scraped/descriptions.csv",
    sep='|',
    header=None,
    names=['URL_name', 'desc_raw'],
    encoding='latin1'
    )

descriptions['description'] = descriptions['desc_raw'].apply(
    lambda x: bs4.BeautifulSoup(str(x), 'lxml').get_text()
    )

descriptions.drop('desc_raw', axis=1, inplace=True)

print(descriptions.shape)

(147851, 2)


# Merge datasets

In [17]:
scraped_data.drop_duplicates(inplace=True)
descriptions.drop_duplicates(inplace=True)

combined = pd.merge(
    left=scraped_data,
    right=descriptions,
    how='left',
    left_on='url',
    right_on='URL_name'
    )

print(combined.shape)

combined.drop_duplicates(subset='name',inplace=True)
combined.reset_index(drop=True, inplace=True)
print(combined.shape)

(129942, 7)
(125787, 7)


# Save

In [21]:
combined.to_pickle('data/scraped/scraped_with_decs.pickle', compression='gzip')